<a href="https://colab.research.google.com/github/Mehty/NLP/blob/master/Text_Generation_using_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import re
import pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
with open(data_path, 'r', encoding = "ISO-8859-1") as f:
  print(f.read())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
# Load date
data_path = '/content/drive/My Drive/Colab Notebooks/data/ner_dataset.csv'
data = pandas.read_csv(data_path, encoding = "ISO-8859-1")
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [14]:
words = data.Word
tags = data.Tag

names = []
for word, tag in zip(words, tags):
  if tag == 'O': continue
  if not tag.endswith('-per'): continue
  if tag == 'B-per':
    names.append(word)
  else:
    names[-1] = names[-1] + ' ' + word

print(len(names))

16990


In [15]:
# Only alphabets, hyphens and spaces allowed.
# Minimum 2 words. (Filters out names like "Bush")
names = [name for name in names if re.match('^[A-Za-z\- ]*$', name)]
names = list(set(names)) # unique
print('Number of names:', len(names))
 
train_data = names[:3600]
test_data = names[3600:4000]
 
print(len(train_data), train_data[:10])
 
all_letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ- $' # $ denotes BOS / EOS
n_letters = len(all_letters)

Number of names: 5631
3600 ['NE WIN', 'Sao Tome', 'Ireju Bares', 'Dennis Weaver', 'Chanderpaul', 'Githongo', 'Kevin James', 'Jean-Baptiste Natama', 'Larry Paul Fidler', 'Franz Klammer']


In [16]:
###############################################################################
## Creating the Network
 
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
 
        ## declare the layers
        # rnn layer 1. input = input + hidden values from previous time step
        self.rnn1 = nn.Linear(input_size+hidden_size, hidden_size) 
        # rnn layer 2. input = hidden values from rnn1 + hidden values from previous time step
        self.rnn2 = nn.Linear(hidden_size + hidden_size, hidden_size) 
        # final layer. output from rnn2 to final output  
        self.output = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
 
    def forward(self, input, hidden):
        hidden1 = F.sigmoid(self.rnn1(torch.cat((input, hidden[0]), 1)))
        # dropout layer (optional) 
        hidden1 = F.dropout(hidden1, p=0.2, training=self.training)
        
        hidden2 = F.sigmoid(self.rnn2(torch.cat((hidden1, hidden[1]), 1)))
        # dropout layer (optional)
        hidden2 = F.dropout(hidden2, p=0.2, training=self.training)
        output = self.softmax(self.output(hidden2))
        return output, (hidden1, hidden2)
 
    def init_hidden(self):
        # initialize hidden layers to 0 at the beginning
        return (Variable(torch.zeros(1, self.hidden_size)), Variable(torch.zeros(1, self.hidden_size)))
 
rnn = RNN(n_letters, 128, n_letters)
criterion = nn.NLLLoss() # negative log likelihood
 
###############################################################################
## Preparing Data for Training
 
# One-hot matrix of first to last letters (+BOS) for input
def input_tensor(name, bos=True):
    input = ('$' + name) if bos else name
    tensor = torch.zeros(len(input), 1, n_letters)
    for idx, letter in enumerate(input):
        tensor[idx][0][all_letters.find(letter)] = 1
    return tensor
 
# Index of first letter to last letter (+EOS) for target
def target_tensor(name):
    target = name + '$'
    letter_indexes = [all_letters.find(letter) for letter in target]
    return torch.LongTensor(letter_indexes)
 
print(input_tensor('Keshav Dhandhania'))
print(target_tensor('Keshav Dhandhania')) 
 

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [0]:
## Training
 
import sys
 
log_interval = 100
learning_rate = 0.01
previous_loss = 100.0
 
def train():
    global learning_rate, previous_loss
    print('Using learning rate: %.5f' % learning_rate)
 
    rnn.train()
    running_loss = 0.0
 
    for ii, name in enumerate(train_data):
        if ii % log_interval == 0:
            print('.', end='')
            sys.stdout.flush()
        input, target = Variable(input_tensor(name)), Variable(target_tensor(name))
 
        rnn.zero_grad()
        hidden = rnn.init_hidden()
 
        loss = 0
        for idx in range(len(name)+1):
            output, hidden = rnn(input[idx], hidden)
            loss += criterion(output, target[idx].unsqueeze(0))
        loss.backward()
 
        # torch.nn.utils.clip_grad_norm(rnn.parameters(), 0.25)
        for p in rnn.parameters():
            p.data.add_(-learning_rate, p.grad.data)
 
        running_loss += loss.item()
    print('')
 
    avg_loss = running_loss / sum(len(name) for name in train_data)
    if previous_loss < avg_loss: learning_rate *= 0.8
    previous_loss = avg_loss
    print('Training loss: %.3f' % avg_loss)
 
###############################################################################
## Testing
 
def test():
    rnn.eval()
 
    running_loss = 0.0
 
    for name in test_data:
        input, target = Variable(input_tensor(name)), Variable(target_tensor(name))
        hidden = rnn.init_hidden()
 
        loss = 0
        for idx in range(len(name)+1):
            output, hidden = rnn(input[idx], hidden)
            loss += criterion(output, target[idx].unsqueeze(0))
 
        running_loss += loss.item()
 
    avg_loss = running_loss / sum(len(name) for name in test_data)
    print('Testing loss: %.3f' % avg_loss)

In [0]:
## Sampling the network
 
from numpy.random import choice
 
max_length = 50
 
# Sample given a starting letter
def sample(prefix=''):
    rnn.eval()
 
    output_string = prefix
    ... # get initial hidden vector
 
    for idx in range(max_length):
        input = Variable(input_tensor(output_string))
        ... # execute one time step of rnn 
 
        if idx < len(prefix): continue # still 'feeding' in the prefix, no need to change output_string
 
        probabilities = ... # calculate probabilities from output
        sample_idx = ... # sample idx between (0, n_letters) using numpy choice
        if sample_idx == n_letters - 1: break # EOS 
 
        letter = ... 
        output_string = ... 
 
    print('Prefix = "%s".' % prefix, 'Generated string =', output_string)
    return output_string
 
# Get multiple samples from one category and multiple starting letters
def samples():
    for i in range(5):
        sample()
    for start_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
        sample(start_letter)